In [1]:
import sys
sys.path.append("..")

import pandas as pd
import re
import nltk
from nltk.stem.snowball import FrenchStemmer

#!python3 -m spacy download fr_core_news_md
import spacy
nlp = spacy.load('fr_core_news_md')

from unidecode import unidecode

pd.set_option('display.max_rows', None)

from quotaclimat.data_analytics.analytics_signataire_charte import (
    bar_plot_nb_of_signataire_per_media, get_summary_statistics)

In [2]:
# Load dataframe
sys.path.append("..")
df_signing = pd.read_csv('../data_public/signataire_charte/generated_on=20221125/signataire_chartejournalismeecologie.csv')


def check_unique_values(df, column_names):
    """
    Print unique values for given column.
    """
    for name in column_names:
        print(f"Number of unique values in {name}:", len(df[name].unique()))

check_unique_values(df_signing, ["Média ou organisation", "Fonction"])

df_signing.head()

Number of unique values in Média ou organisation: 956
Number of unique values in Fonction: 591


,Nom,Prénom,Fonction,Média ou organisation
0,Aballain,Olivier,Directeur des formations au journalisme,ESJ Lille
1,Abalo,André,JRI,France Télévision
2,Abbiateci,Jean,Journaliste,NaN
3,Aboudou,Djamiou,Journaliste,L’Emissaire
4,Acar,Anne-Lucie,Journaliste,Free-lance


In [3]:
def apply_stemmer(df, column_name) -> pd.DataFrame: 
    """
    Apply a Porter Stemmer to the given column of a dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to be transformed.
    column_name : str
        The name of the column to apply the Porter Stemmer.

    Returns
    -------
    The entire dataframe, with the new column.
    """
    # New column name
    new_column_name = f"Stemmed {column_name}"
    # Unidecode to remove accent
    #df[new_column_name] = [unidecode(sentence) if pd.notna(sentence) \
    #    else sentence for sentence in df[column_name]]
    # Apply porter stemmer
    ps = FrenchStemmer()
    df[new_column_name] = [[ps.stem(token) for token in sentence.split(" ")] \
        if pd.notna(sentence) else sentence for sentence in df[column_name]]
    # Join stemmed words
    df[new_column_name] = [" ".join(sentence).capitalize() \
        if type(sentence) == list else sentence for sentence in df[new_column_name]]

    return df


def apply_lemmatizer(df, column_name) -> pd.DataFrame: 
    """
    Lemmatize the given column of a dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to be transformed.
    column_name : str
        The name of the column to apply the lemmatization.

    Returns
    -------
    The entire dataframe, with the new column.
    """
    # New column name
    new_column_name = f"Lemmatized {column_name}"

    # Apply porter stemmer
    df[new_column_name] = [[token.lemma_ for token in nlp(sentence)] \
        if pd.notna(sentence) else sentence for sentence in df[column_name]]
    # Join stemmed words
    df[new_column_name] = [" ".join(sentence).capitalize() \
        if type(sentence) == list else sentence for sentence in df[new_column_name]]

    # Clean media with REGEX
    df[new_column_name] = df[new_column_name] \
        .str.replace(r"[Ff]rance.?3.*", "France 3") \
        .str.replace(r"[Ff]rance.?télévisions?|[Ff]rance.?[Tt][Vv].?", "France TV") \
        .str.replace(r"[Ff]rance.?info.*", "France info") \
        .str.replace(r".*[Ii]nd[ée]pend.*", "Indépendant") \
        .str.replace(r"Free - lancer", "Freelance")

    return df

for name in ["Fonction", "Média ou organisation"]:
    df_signing = apply_stemmer(df_signing, column_name=name)
    df_signing = apply_lemmatizer(df_signing, column_name=name)
df_signing.head()

/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipykernel_42638/3909630348.py:59: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"[Ff]rance.?3.*", "France 3") \
/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipykernel_42638/3909630348.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"[Ff]rance.?télévisions?|[Ff]rance.?[Tt][Vv].?", "France TV") \
/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipykernel_42638/3909630348.py:61: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"[Ff]rance.?info.*", "France info") \
/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipykernel_42638/3909630348.py:62: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r".*[Ii]nd[ée]pend.*", "Indépendant") \
/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipyker

,Nom,Prénom,Fonction,Média ou organisation,Stemmed Fonction,Lemmatized Fonction,Stemmed Média ou organisation,Lemmatized Média ou organisation
0,Aballain,Olivier,Directeur des formations au journalisme,ESJ Lille,Directeur de format au journal,Directeur de formation au journalisme,Esj lill,Esj lille
1,Abalo,André,JRI,France Télévision,Jri,Jri,Franc télévis,France TV
2,Abbiateci,Jean,Journaliste,NaN,Journal,Journaliste,NaN,NaN
3,Aboudou,Djamiou,Journaliste,L’Emissaire,Journal,Journaliste,L’emissair,L’ emissaire
4,Acar,Anne-Lucie,Journaliste,Free-lance,Journal,Journaliste,Free-lanc,Freelance


In [10]:
column_to_check = [
    "Stemmed Fonction", "Lemmatized Fonction", 
    "Stemmed Média ou organisation", "Lemmatized Média ou organisation"
]
check_unique_values(df_signing, column_to_check)

df_signing.groupby("Lemmatized Fonction").count().sort_values("Nom", ascending=False).head(50)

#df_signing['Lemmatized Média ou organisation'].value_counts()

Number of unique values in Stemmed Fonction: 525
Number of unique values in Lemmatized Fonction: 525
Number of unique values in Stemmed Média ou organisation: 892
Number of unique values in Lemmatized Média ou organisation: 851


,Nom,Prénom,Fonction,Média ou organisation,Stemmed Fonction,Stemmed Média ou organisation,Lemmatized Média ou organisation
Lemmatized Fonction,,,,,,,
Journaliste,587,587,587,564,587,564,564
Rédacteur en chef,81,81,81,81,81,81,81
Indépendant,54,54,54,46,54,46,46
Journaliste pigist,24,24,24,19,24,19,19
Journaliste rédacteur,13,13,13,13,13,13,13
Président,11,11,11,11,11,11,11
Rédacteur en chef adjoint,11,11,11,11,11,11,11
Pigiste,11,11,11,10,11,10,10
Reporter,11,11,11,11,11,11,11


In [5]:
import plotly.express as px


def bar_plot_nb_of_signataire_per_media(df, column):
    count_medias = (
        df.groupby(column).count().sort_values("Nom", ascending=False)
    )
    top_n = 50
    fig = px.bar(
        count_medias.reset_index().head(top_n),
        x=column,
        y="Nom",
    )

    fig.update_xaxes(tickangle=-45, title=None)
    fig.update_yaxes(title="Nombre de personne signataire")
    fig.update_layout(
        margin={"b": 100},
        title="Nombre de signataire de la charte journalisme ecologie par média, top %s"
        % top_n,
    )
    return fig

bar_plot_nb_of_signataire_per_media(df_signing, column="Lemmatized Média ou organisation")

In [6]:
bar_plot_nb_of_signataire_per_media(df_signing, column="Stemmed Média ou organisation")

In [7]:
bar_plot_nb_of_signataire_per_media(df_signing, column="Lemmatized Fonction")

In [8]:
[token.lemma_ for token in nlp("rédacteur rédactrice, redacteur redactrice")]

['rédacteur', 'rédacteur', ',', 'redacteur', 'redactrice']

In [9]:
doc = nlp("rédacteur rédactrice, redacteur redactrice directeur directrice")
print([(sent.text) for sent in doc.sents])
print([(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_) for word in doc])

['rédacteur rédactrice, redacteur redactrice directeur directrice']
[('rédacteur', 5015996011091846609, 'rédacteur', 92, 'NOUN', 92, 'NOUN'), ('rédactrice', 5015996011091846609, 'rédacteur', 92, 'NOUN', 92, 'NOUN'), (',', 2593208677638477497, ',', 97, 'PUNCT', 97, 'PUNCT'), ('redacteur', 4160570345990072831, 'redacteur', 92, 'NOUN', 92, 'NOUN'), ('redactrice', 16912174967706751488, 'redactrice', 92, 'NOUN', 92, 'NOUN'), ('directeur', 11878821933488635175, 'directeur', 92, 'NOUN', 92, 'NOUN'), ('directrice', 17596145772598196135, 'directrice', 92, 'NOUN', 92, 'NOUN')]
